In [5]:
import pandas as pd
def read_conll(filename):
    df = pd.read_csv(filename,
                    sep = '\t', header = None, keep_default_na = False,
                    names = ['words', 'labels'], skip_blank_lines = False)
    df['sentence_id'] = (df.words == '').cumsum()
    return df[df.words != '']

In [ ]:
train_df = read_conll('../Datasets/web_datasets/sig_train.txt')
test_df = read_conll('../Datasets/web_datasets/sig_test.txt')
print(test_df)

In [7]:
data = [[train_df['sentence_id'].nunique(), test_df['sentence_id'].nunique()]]
pd.DataFrame(data, columns=["Train", "Test"])

,Train,Test
0,46364,4365


In [8]:
train_args = {
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'sliding_window': True,
    'max_seq_length': 64,
    'num_train_epochs': 5,
    'train_batch_size': 32,
    'fp16': True,
    'output_dir': '/outputs/',
}

In [9]:
from simpletransformers.ner import NERModel
from transformers import AutoTokenizer
import pandas as pd
import logging

logging.basicConfig(level=logging.DEBUG)
transformers_logger = logging.getLogger('transformers')
transformers_logger.setLevel(logging.WARNING)

# We use the bert base cased pre-trained model.
tokenizer = AutoTokenizer.from_pretrained('bert-base-chinese')
model = NERModel('bert', 'bert-base-chinese', args=train_args)

# Train the model, there is no development or validation set for this dataset
# https://simpletransformers.ai/docs/tips-and-tricks/#using-early-stopping
model.train_model(train_df)

# Evaluate the model in terms of accuracy score
result, model_outputs, preds_list = model.eval_model(test_df)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /bert-base-chinese/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /bert-base-chinese/resolve/main/config.json HTTP/1.1" 200 0
Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializin

  0%|          | 0/21 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/1449 [00:00<?, ?it/s]

C:\Users\asdf5\Desktop\Named-Entity-Recognition\venv\Lib\site-packages\torch\optim\lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


...

Running Epoch 4 of 5:   0%|          | 0/1449 [00:00<?, ?it/s]

INFO:simpletransformers.ner.ner_model: Training of bert model complete. Saved to /outputs/.
INFO:simpletransformers.ner.ner_model: Converting to features started.


  0%|          | 0/9 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/546 [00:00<?, ?it/s]

INFO:simpletransformers.ner.ner_model:{'eval_loss': 0.037941540167992324, 'precision': 0.9450171821305842, 'recall': 0.9526558891454965, 'f1_score': 0.9488211615871189}


In [15]:
strs = """曾被譽為牛樟芝大王的劉威甫，2016年起用直銷手法，招攬會員投資培植牛樟芝，誆稱每單位保證金7萬5元，期滿還可全額領回保證金，涉嫌吸金30億，檢方訊問後，董事長劉威甫100萬交保，台灣分公司總經理莊立平則是諭令200萬交保，遭限制出境。平頭、白髮，遭檢調人員帶回北檢，他是被封為牛樟芝大王的台商劉威甫，2014年在中國風光成立中國珍菌堂集團成了董事長，如今卻是檢調人員頭號鎖定對象。集團總經理莊立平和公司掛名負責人張桂銘也都到北檢複訊，就是因為他們宣稱以「椴木培植法」培育牛樟芝，甚至還能直接拿商品買賣，利用直銷方式吸引會員。珍菌堂廣告：「建設有牛樟樹，種苗培育基地 。」集團經理莊立平在大陸認識劉威甫後，說服董事長在2016年回台開設分公司，還誆稱牛樟芝能治癌症，做噱頭，吸引3萬人成為會員，更誆稱期滿後本金可全數退回，短短兩年涉嫌吸金30億。記者吳欣倫：「珍菌堂他賣的不只是成品，還主打可以種植牛樟芝，讓大家成為小農，也就是成為養菌培植戶，只要每一平方公尺，就能來培植而且可以形成所謂的互利互助概念，更主打說只要投資花7萬5千元，甚至你只要每周付7百元，你就可以拿回所謂的保證金。」珍菌堂董事長劉威甫：「我們投入一些開發資金也好，都是公司這個行業前所未見。」但2018年開始，台灣投資人陸續收不到紅利，莊立平似乎怕東窗事發，把共享獎金設為浮動制 ，看牛樟液銷量多少才依比例發放，投資人氣得提告。大樓保全：「搬很久了，好久了 至少有5年了。」而總經理莊立平遭指控是全案主謀，他全盤否認 ，只說自己是珍菌堂海外經銷商，最後被檢方諭令200萬交保並遭限制出境，董事長劉威甫則供稱，所有買賣合約都是總經理負責，與他無關，最後被諭令百萬交保。●東森新聞關心您
不良行為，請勿模仿（封面圖／東森新聞）"""
samples = [' '.join(strs)]
predictions, _ = model.predict(samples)
for idx, sample in enumerate(samples):
  print('{}: '.format(idx))
  for word in predictions[idx]:
    print('{}'.format(word))

INFO:simpletransformers.ner.ner_model: Converting to features started.


  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

0: 
{'曾': 'O'}
{'被': 'O'}
{'譽': 'O'}
{'為': 'O'}
{'牛': 'O'}
{'樟': 'O'}
{'芝': 'O'}
{'大': 'O'}
{'王': 'O'}
{'的': 'O'}
{'劉': 'B-PER'}
{'威': 'I-PER'}
{'甫': 'I-PER'}
{'，': 'O'}
{'2': 'O'}
{'0': 'O'}
{'1': 'O'}
{'6': 'O'}
{'年': 'O'}
{'起': 'O'}
{'用': 'O'}
{'直': 'O'}
{'銷': 'O'}
{'手': 'O'}
{'法': 'O'}
{'，': 'O'}
{'招': 'O'}
{'攬': 'O'}
{'會': 'O'}
{'員': 'O'}
{'投': 'O'}
{'資': 'O'}
{'培': 'O'}
{'植': 'O'}
{'牛': 'O'}
{'樟': 'O'}
{'芝': 'O'}
{'，': 'O'}
{'誆': 'O'}
{'稱': 'O'}
{'每': 'O'}
{'單': 'O'}
{'位': 'O'}
{'保': 'O'}
{'證': 'O'}
{'金': 'O'}
{'7': 'O'}
{'萬': 'O'}
{'5': 'O'}
{'元': 'O'}
{'，': 'O'}
{'期': 'O'}
{'滿': 'O'}
{'還': 'O'}
{'可': 'O'}
{'全': 'O'}
{'額': 'O'}
{'領': 'O'}
{'回': 'O'}
{'保': 'O'}
{'證': 'O'}
{'金': 'O'}
